リスト内の1の和  
e.g.  
[1,1,0,0,0,0,1,0,0,0] -> 3  

http://peterroelants.github.io/posts/rnn_implementation_part01/

In [1]:
!rm -rf tmp/tensorflow_log/*

import tensorflow as tf
import numpy as np
import random


num_of_input_nodes = 1
num_of_hidden_nodes = 80
num_of_output_nodes = 1
length_of_sequences = 10
num_of_training_epochs = 5000
size_of_mini_batch = 100
num_of_prediction_epochs = 100
learning_rate = 0.01
forget_bias = 0.8
num_of_sample = 1000
num_layers = 1


def get_batch(batch_size, X, t):
    rnum = [random.randint(0, len(X) - 1) for x in range(batch_size)]
    xs = np.array([[[y] for y in list(X[r])] for r in rnum])
    ts = np.array([t[r] for r in rnum])
    return xs, ts


def create_data(num_of_samples, sequence_len):
    X = np.zeros((num_of_samples, sequence_len))
    t = np.zeros((num_of_samples, 11))
    for row_idx in range(num_of_samples):
        X[row_idx, :] = np.random.randint(2, size=sequence_len)
        sum = np.sum(X[row_idx, :])
        t[row_idx, int(sum)] = 1
    return X, t


def unpack_sequence(tensor):
    return tf.unpack(tf.transpose(tensor, perm=[1, 0, 2]))

def pack_sequence(sequence):
    return tf.transpose(tf.pack(sequence), perm=[1, 0, 2])

def inference(input_ph):
    with tf.name_scope("inference") as scope:
        in_size = num_of_hidden_nodes
        out_size = 11
        weight = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1))
        bias = tf.Variable(tf.constant(0.1, shape=[out_size]))
        
        istate = tf.zeros((size_of_mini_batch, num_of_hidden_nodes * 2))
        init_state = tf.Variable(istate, dtype=tf.float32, name="init_state")
       
        network = tf.nn.rnn_cell.LSTMCell(num_of_hidden_nodes)
        network = tf.nn.rnn_cell.DropoutWrapper(network, output_keep_prob=0.5)
        network = tf.nn.rnn_cell.MultiRNNCell([network] * num_layers)
        inputs =  unpack_sequence(input_ph)
        
        rnn_output, states_op = tf.nn.rnn(network,inputs,dtype=tf.float32)
        #rnn_output = pack_sequence(rnn_output)
        #state_op = pack_sequence(states_op)
        output_op = tf.nn.softmax(tf.matmul(rnn_output[-1], weight) + bias)

 
        w1_hist = tf.histogram_summary("weights", weight)
        b1_hist = tf.histogram_summary("biases", bias)
        output_hist = tf.histogram_summary("output",  output_op)
        results = [weight, bias]
        return output_op, states_op, results


def loss(output_op, supervisor_ph):
    with tf.name_scope("loss") as scope:
        loss_op = - tf.reduce_sum(supervisor_ph * tf.log(output_op))
        tf.scalar_summary("loss", loss_op)
        return loss_op


def training(loss_op):
    with tf.name_scope("training") as scope:
        training_op = optimizer.minimize(loss_op)
        return training_op

def accuracy(output_op, supervisor_ph):
    with tf.name_scope("accuracy") as scope:
        correct_prediction = tf.equal(tf.argmax(output_op,1), tf.argmax(supervisor_ph,1))
        accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tf.scalar_summary("accuracy", accuracy_op)
        return accuracy_op

def calc_accuracy(accuracy_opp, X, t):
    inputs, targets = get_batch(len(X_train), X_train, t_train)
    pred_dict = {
        input_ph:  inputs,
        supervisor_ph: targets
    }
    accurecy = sess.run(accuracy_op, feed_dict=pred_dict)
    print(accurecy)



random.seed(0)
np.random.seed(0)
tf.set_random_seed(0)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

X, t = create_data(num_of_sample, length_of_sequences)
test_num = int(num_of_sample*0.3)
X_test = X[:test_num]
t_test = t[:test_num]
X_train = X[test_num:]
t_train = t[test_num:]
print("テストデータ数: ", len(X_test))
print("トレーニングデータ数: ", len(X_train))

with tf.Graph().as_default():
    input_ph = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, 11], name="supervisor")

    output_op, states_op, datas_op = inference(input_ph)
    loss_op = loss(output_op, supervisor_ph)
    training_op = training(loss_op)
    accuracy_op = accuracy(output_op, supervisor_ph)

    summary_op = tf.merge_all_summaries()
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        saver = tf.train.Saver()
        summary_writer = tf.train.SummaryWriter("tmp/tensorflow_log", graph=sess.graph)
        sess.run(init)

        for epoch in range(num_of_training_epochs):
            inputs, supervisors = get_batch(size_of_mini_batch, X_train, t_train)
            train_dict = {
                input_ph:   inputs,
                supervisor_ph: supervisors
            }
            sess.run(training_op, feed_dict=train_dict)

            if (epoch) % 100 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                print("train#{}, loss: {}".format(epoch, train_loss))
                summary_writer.add_summary(summary_str, epoch)
                if (epoch) % 500 == 0:
                    calc_accuracy(output_op, X_test, t_test)
        calc_accuracy(output_op, X_test, t_test)
        datas = sess.run(datas_op)
        saver.save(sess, "model.ckpt")

テストデータ数:  300
トレーニングデータ数:  700
train#0, loss: 218.46011352539062
0.187143
train#100, loss: 100.57823181152344
train#200, loss: 112.99855041503906
train#300, loss: 20.311420440673828
train#400, loss: 11.360490798950195
train#500, loss: 6.569569110870361
0.98
train#600, loss: 3.9332756996154785
train#700, loss: 4.560934066772461
train#800, loss: 4.625699520111084
train#900, loss: 3.8007025718688965
train#1000, loss: 1.9310370683670044
0.991429
train#1100, loss: 1.556487798690796
train#1200, loss: 0.9503235220909119
train#1300, loss: 1.6706324815750122
train#1400, loss: 2.4234113693237305
train#1500, loss: 1.1390825510025024
0.997143
train#1600, loss: 0.8369040489196777
train#1700, loss: 0.6445013284683228
train#1800, loss: 1.3029805421829224
train#1900, loss: 0.8373410105705261
train#2000, loss: 0.4186079502105713
0.997143
train#2100, loss: 0.2931709587574005
train#2200, loss: 0.34993115067481995
train#2300, loss: 0.8678434491157532
train#2400, loss: 0.7747615575790405
train#2500, loss: 